#### 2. Optimización escenas de doblaje:

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben. No es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que se gasto por los
servicios de los actores de doblaje sea el menor posible.

In [1]:
import os.path
import pandas as pd
import numpy as np
import random 
import math
import copy 

import warnings
warnings.filterwarnings('ignore')

In [2]:
#Lee el excel
path = "./datosDoblaje.xlsx"
norm_path = os.path.normpath(path) 
datosDoblaje = pd.read_excel(norm_path)

##### Primera solución -> Algoritmo Voraz
No será la mejor solución pero nos proporcionará una buena primera solución

In [3]:
diaFinal = list()
posInterAnt = list()
posInterPost = list()

def optimizaOrdenDoblaje(diaOrden, actores, escena, matrizOcupacion):
    if(escena+1 == 31):
        print('YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS')
        #print('/////////////////////////////////////////////')
        #print('///////// Se ha tardado',diaOrden,'dias //////////////')
        #print('/////////////////////////////////////////////')
        diaFinal.append(diaOrden)
    else:
        total = 0
        for i in range(actores):
            actorPos = 'ACT' + str(i+1)
            participacionesActor = matrizOcupacion[i][diaOrden]
            
            if(datosDoblaje.iloc[escena][actorPos]== 1):
                if(participacionesActor < 6):
                    matrizOcupacion[i][diaOrden] = matrizOcupacion[i][diaOrden] + 1
                    total = total + 1
                    #print('El dia',diaOrden,'El actor',actorPos,'puede trabajar en la escena',escena+1)
                    #print('Este actor ha trabajado hoy en',matrizOcupacion[i][diaOrden],'escenas')
                #else:
                    #print('El dia',diaOrden,'El actor',actorPos,'NO puede trabajar en la escena',escena+1)
                    #print('Este actor ha trabajado hoy en',matrizOcupacion[i][diaOrden],'escenas')
        if(total == datosDoblaje.iloc[escena]['Total']):
            #print('////////////////////////////////////////////////////////////')
            #print('La escena',escena+1,'se va a llevar a cabo el dia',diaOrden)
            #print('////////////////////////////////////////////////////////////')
            optimizaOrdenDoblaje(diaOrden, actores, escena+1, matrizOcupacion)
        else:
            #print('**************************************************************')
            #print('La escena',escena+1,'NO se va a llevar a cabo el dia',diaOrden)
            #print('**************************************************************')
            optimizaOrdenDoblaje(diaOrden+1, actores, escena, matrizOcupacion)

In [4]:
actores  = 10
escena  = 0
diaOrden =1

matrizOcupacion = np.zeros((10, 40))

optimizaOrdenDoblaje(diaOrden, actores, escena, matrizOcupacion)

YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS


#### Segunda solución -> Algoritmos Heurísticos -> Busqueda Tabu

In [5]:
def genera_vecina_aleatorio(solucion_referencia):
    #Coge dos nodos aleatorios y los intercambia
    i,j = sorted(random.sample( range(0,len(solucion_referencia)) , 2))
    #Modifica el dataframe cambiando esos dos nodos de posicion
    cambia_orden_nodos(i,j)
    
    posInterAnt.append(i)
    posInterPost.append(j)
    
    aux = solucion_referencia[i]
    solucion_referencia[i] = solucion_referencia[j]
    solucion_referencia[j] = aux
    return solucion_referencia

In [6]:
def rollBack(i, j, solucion_referencia):
    cambia_orden_nodos(i,j)
    
    aux = solucion_referencia[i]
    solucion_referencia[i] = solucion_referencia[j]
    solucion_referencia[j] = aux
    
    return solucion_referencia

In [7]:
def cambia_orden_nodos(nodo1, nodo2):
    aux = datosDoblaje.iloc[nodo1]
    datosDoblaje.iloc[nodo1] = datosDoblaje.iloc[nodo2]
    datosDoblaje.iloc[nodo2] = aux

In [8]:
def probabilidad(T,d):
    if random.random() <  math.exp( -1*d / T)  :
        return True
    else:
        return False

In [9]:
#Funcion de descenso de temperatura
def bajar_temperatura(T):
    return T*0.99

In [10]:
def recorrido_simulado(TEMPERATURA):
    solucion_referencia = list()
    #Se utiliza  como solucion de referencia la calculada previamente
    for i in range(30): 
        solucion_referencia.append(i)
    matrizOcupacion = np.zeros((10, 40))
    optimizaOrdenDoblaje(1, 10, 0, matrizOcupacion)
    distancia_referencia = diaFinal[-1]
      
    mejor_solucion = list()
    mejor_distancia = 1000
    
    N=0
    while TEMPERATURA > .0001:
        N+=1
        #Genera una solución vecina
        vecina = genera_vecina_aleatorio(solucion_referencia)
        
        #Calcula el numero de dias
        matrizOcupacion = np.zeros((10, 40))
        optimizaOrdenDoblaje(1, 10, 0, matrizOcupacion)
        distancia_vecina = diaFinal[-1]
        
        #Si es la mejor solución de todas se guarda(siempre!!!)
        if distancia_vecina < mejor_distancia:
            print('Se ha tardado', distancia_vecina,'dias')
            print('La mejor solucion es', vecina)
            mejor_solucion = vecina
            mejor_distancia = distancia_vecina
            
        if distancia_vecina < distancia_referencia or probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina) ) :
            print('Se ha tardado', distancia_vecina,'dias')
            print('No mejor solucion es', vecina)
            
            if(distancia_referencia < distancia_vecina):
                solucion_referencia = rollBack(posInterAnt[-1], posInterPost[-1], solucion_referencia)
                vecina = genera_vecina_aleatorio(solucion_referencia)
                
                matrizOcupacion = np.zeros((10, 40))
                optimizaOrdenDoblaje(1, 10, 0, matrizOcupacion)
                distancia_vecina = diaFinal[-1]
                
                print('Hemos vuelto a la solucion anterior')
                print('Se ha tardado', distancia_vecina,'dias')
                print('ROLLBACK solucion', vecina)
            
            solucion_referencia = copy.deepcopy(vecina)
            distancia_referencia = distancia_vecina
        
        TEMPERATURA = bajar_temperatura(TEMPERATURA)
    
    print("La mejor solución encontrada es " , end="")
    print(mejor_solucion)
    print("con una distancia total de " , end="")
    print(mejor_distancia)
    return mejor_solucion

In [11]:
solucion_referencia = list()
    #Se utiliza  como solucion de referencia la calculada previamente
for i in range(30): 
    solucion_referencia.append(i)
    
sol  = recorrido_simulado(10000000)

YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
La mejor solucion es [0, 16, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 1, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
Se ha tardado 4 dias
No mejor solucion es [0, 16, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 1, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [0, 16, 2, 3, 4, 5, 6, 7, 8, 9, 15, 11, 12, 13, 14, 10, 1, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [0, 16, 2, 3, 4, 5, 6, 7, 11, 9, 15, 8, 12, 13, 14, 10, 1, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 5 dias
No mejor solucion es [0, 16, 2, 3, 4, 5, 6, 23, 11, 9, 15, 8, 12, 13, 14, 10, 1, 17, 18, 19, 20, 21, 22, 7, 24, 25, 26, 27, 28, 29]
YA HEMOS ACABADO DE ORGAN

Se ha tardado 4 dias
No mejor solucion es [4, 14, 11, 22, 5, 20, 21, 9, 1, 26, 29, 8, 25, 13, 17, 27, 2, 10, 19, 12, 16, 3, 18, 6, 23, 7, 0, 15, 24, 28]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [4, 14, 11, 22, 5, 20, 21, 9, 1, 26, 29, 8, 25, 13, 6, 27, 2, 10, 19, 12, 16, 3, 18, 17, 23, 7, 0, 15, 24, 28]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [4, 14, 11, 22, 5, 20, 21, 9, 1, 26, 29, 8, 25, 13, 6, 27, 2, 10, 12, 19, 16, 3, 18, 17, 23, 7, 0, 15, 24, 28]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [4, 14, 11, 22, 12, 20, 21, 9, 1, 26, 29, 8, 25, 13, 6, 27, 2, 10, 5, 19, 16, 3, 18, 17, 23, 7, 0, 15, 24, 28]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [4, 14, 11, 22, 12, 20, 21, 9, 8, 26, 29, 1, 25, 13, 6, 27, 2, 10, 5, 19, 16, 3, 18, 17, 23, 7, 0, 15, 24, 28]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No m

YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [8, 14, 13, 22, 18, 16, 17, 10, 21, 29, 5, 25, 7, 2, 1, 9, 23, 3, 11, 19, 12, 0, 4, 15, 28, 6, 26, 20, 24, 27]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [8, 14, 13, 22, 18, 16, 17, 10, 21, 29, 5, 25, 7, 15, 1, 9, 23, 3, 11, 19, 12, 0, 4, 2, 28, 6, 26, 20, 24, 27]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [8, 14, 13, 22, 18, 7, 17, 10, 21, 29, 5, 25, 16, 15, 1, 9, 23, 3, 11, 19, 12, 0, 4, 2, 28, 6, 26, 20, 24, 27]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [8, 14, 13, 22, 18, 7, 17, 10, 9, 29, 5, 25, 16, 15, 1, 21, 23, 3, 11, 19, 12, 0, 4, 2, 28, 6, 26, 20, 24, 27]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [8, 14, 13, 22, 18, 7, 17, 10, 9, 28, 5, 25, 16, 15, 1, 21, 23, 3, 11, 19, 12, 0, 4, 2, 29, 6, 26, 20, 24, 27]
YA HEMOS ACABADO DE ORGAN

YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [27, 0, 2, 19, 23, 22, 6, 26, 29, 14, 16, 1, 7, 15, 17, 13, 20, 5, 11, 12, 28, 10, 21, 18, 24, 4, 25, 8, 3, 9]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [27, 0, 2, 19, 23, 22, 16, 26, 29, 14, 6, 1, 7, 15, 17, 13, 20, 5, 11, 12, 28, 10, 21, 18, 24, 4, 25, 8, 3, 9]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [27, 24, 2, 19, 23, 22, 16, 26, 29, 14, 6, 1, 7, 15, 17, 13, 20, 5, 11, 12, 28, 10, 21, 18, 0, 4, 25, 8, 3, 9]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [27, 24, 2, 15, 23, 22, 16, 26, 29, 14, 6, 1, 7, 19, 17, 13, 20, 5, 11, 12, 28, 10, 21, 18, 0, 4, 25, 8, 3, 9]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [27, 24, 2, 15, 23, 22, 16, 26, 17, 14, 6, 1, 7, 19, 29, 13, 20, 5, 11, 12, 28, 10, 21, 18, 0, 4, 25, 8, 3, 9]
YA HEMOS ACABADO DE ORGAN

YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [22, 21, 7, 19, 1, 4, 27, 25, 8, 2, 9, 29, 15, 28, 20, 12, 23, 11, 17, 26, 5, 24, 13, 10, 3, 6, 14, 18, 0, 16]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [22, 21, 7, 19, 27, 4, 1, 25, 8, 2, 9, 29, 15, 28, 20, 12, 23, 11, 17, 26, 5, 24, 13, 10, 3, 6, 14, 18, 0, 16]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [22, 21, 7, 19, 27, 4, 1, 25, 14, 2, 9, 29, 15, 28, 20, 12, 23, 11, 17, 26, 5, 24, 13, 10, 3, 6, 8, 18, 0, 16]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [5, 21, 7, 19, 27, 4, 1, 25, 14, 2, 9, 29, 15, 28, 20, 12, 23, 11, 17, 26, 22, 24, 13, 10, 3, 6, 8, 18, 0, 16]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [5, 21, 7, 19, 27, 4, 15, 25, 14, 2, 9, 29, 1, 28, 20, 12, 23, 11, 17, 26, 22, 24, 13, 10, 3, 6, 8, 18, 0, 16]
YA HEMOS ACABADO DE ORGAN

...

No mejor solucion es [1, 29, 5, 8, 28, 16, 2, 26, 12, 27, 23, 21, 19, 0, 25, 6, 13, 3, 18, 10, 11, 24, 7, 20, 22, 4, 17, 14, 9, 15]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [1, 29, 5, 8, 28, 16, 2, 26, 12, 27, 23, 21, 19, 0, 25, 6, 13, 3, 18, 10, 11, 24, 7, 20, 17, 4, 22, 14, 9, 15]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [1, 29, 5, 8, 28, 16, 2, 26, 12, 20, 23, 21, 19, 0, 25, 6, 13, 3, 18, 10, 11, 24, 7, 27, 17, 4, 22, 14, 9, 15]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [1, 29, 5, 8, 28, 16, 2, 26, 12, 20, 23, 25, 19, 0, 21, 6, 13, 3, 18, 10, 11, 24, 7, 27, 17, 4, 22, 14, 9, 15]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [1, 29, 5, 8, 28, 16, 2, 26, 13, 20, 23, 25, 19, 0, 21, 6, 12, 3, 18, 10, 11, 24, 7, 27, 17, 4, 22, 14, 9, 15]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [1, 

Se ha tardado 4 dias
No mejor solucion es [12, 29, 5, 18, 14, 9, 17, 16, 1, 13, 6, 25, 2, 28, 0, 20, 10, 3, 8, 22, 27, 23, 7, 26, 24, 11, 21, 4, 15, 19]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [12, 29, 5, 18, 14, 9, 17, 16, 1, 13, 7, 25, 2, 28, 0, 20, 10, 3, 8, 22, 27, 23, 6, 26, 24, 11, 21, 4, 15, 19]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [12, 29, 5, 18, 14, 9, 17, 16, 1, 13, 7, 11, 2, 28, 0, 20, 10, 3, 8, 22, 27, 23, 6, 26, 24, 25, 21, 4, 15, 19]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [12, 29, 5, 18, 14, 9, 17, 16, 1, 13, 7, 11, 2, 28, 0, 20, 10, 3, 8, 4, 27, 23, 6, 26, 24, 25, 21, 22, 15, 19]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [12, 29, 5, 18, 14, 9, 17, 16, 1, 25, 7, 11, 2, 28, 0, 20, 10, 3, 8, 4, 27, 23, 6, 26, 24, 13, 21, 22, 15, 19]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No m

No mejor solucion es [14, 3, 11, 28, 1, 22, 10, 29, 9, 16, 0, 17, 7, 18, 8, 19, 12, 24, 4, 6, 25, 20, 5, 21, 15, 26, 23, 13, 27, 2]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [21, 3, 11, 28, 1, 22, 10, 29, 9, 16, 0, 17, 7, 18, 8, 19, 12, 24, 4, 6, 25, 20, 5, 14, 15, 26, 23, 13, 27, 2]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [21, 3, 11, 28, 1, 22, 10, 29, 9, 16, 0, 17, 7, 4, 8, 19, 12, 24, 18, 6, 25, 20, 5, 14, 15, 26, 23, 13, 27, 2]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [20, 3, 11, 28, 1, 22, 10, 29, 9, 16, 0, 17, 7, 4, 8, 19, 12, 24, 18, 6, 25, 21, 5, 14, 15, 26, 23, 13, 27, 2]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [20, 3, 11, 28, 1, 22, 25, 29, 9, 16, 0, 17, 7, 4, 8, 19, 12, 24, 18, 6, 10, 21, 5, 14, 15, 26, 23, 13, 27, 2]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [20,

YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [28, 4, 7, 10, 12, 29, 25, 18, 9, 15, 26, 17, 0, 13, 16, 14, 1, 19, 20, 27, 3, 2, 8, 22, 21, 11, 24, 5, 23, 6]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [28, 4, 7, 10, 12, 29, 25, 18, 9, 15, 26, 17, 0, 13, 16, 14, 1, 23, 20, 27, 3, 2, 8, 22, 21, 11, 24, 5, 19, 6]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [28, 4, 7, 10, 12, 29, 25, 18, 19, 15, 26, 17, 0, 13, 16, 14, 1, 23, 20, 27, 3, 2, 8, 22, 21, 11, 24, 5, 9, 6]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [28, 4, 7, 10, 1, 29, 25, 18, 19, 15, 26, 17, 0, 13, 16, 14, 12, 23, 20, 27, 3, 2, 8, 22, 21, 11, 24, 5, 9, 6]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [28, 4, 7, 10, 1, 29, 25, 18, 19, 15, 26, 17, 0, 13, 16, 14, 12, 23, 20, 27, 22, 2, 8, 3, 21, 11, 24, 5, 9, 6]
YA HEMOS ACABADO DE ORGAN

YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [20, 21, 12, 8, 1, 2, 19, 9, 25, 10, 3, 4, 0, 6, 11, 14, 15, 7, 22, 27, 24, 5, 23, 26, 17, 18, 28, 16, 13, 29]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [20, 21, 12, 8, 1, 2, 19, 5, 25, 10, 3, 4, 0, 6, 11, 14, 15, 7, 22, 27, 24, 9, 23, 26, 17, 18, 28, 16, 13, 29]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [20, 21, 12, 8, 1, 2, 19, 5, 25, 10, 3, 4, 0, 6, 11, 14, 15, 7, 22, 27, 24, 9, 26, 23, 17, 18, 28, 16, 13, 29]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [20, 21, 12, 8, 1, 2, 19, 5, 25, 10, 28, 4, 0, 6, 11, 14, 15, 7, 22, 27, 24, 9, 26, 23, 17, 18, 3, 16, 13, 29]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [20, 21, 12, 8, 1, 2, 19, 5, 25, 10, 28, 14, 0, 6, 11, 4, 15, 7, 22, 27, 24, 9, 26, 23, 17, 18, 3, 16, 13, 29]
YA HEMOS ACABADO DE ORGAN